# Test of the NEB Tm Calculator ([tmcalculator.neb.com](https://tmcalculator.neb.com/#!/batch))



In [10]:
import csv
primerlist = []
with open('tmcalc_batch.txt', newline='') as csvfile:
    neb_file_reader = csv.reader(csvfile, delimiter='\t')
    for row in neb_file_reader:
        name, sequence, nebtm, *rest = row
        primerlist.append([name, sequence, nebtm])
print(len(primerlist))

200


In [28]:
from Bio.SeqUtils import MeltingTemp as mt
from prettytable import PrettyTable
t = PrettyTable(['Name', 'Sequence', "neb", "bp", "delta"])

for name, primer, nebtm in primerlist:
    # For Phusion, use this settings
    bptm = mt.Tm_NN(
            primer,
            nn_table=mt.DNA_NN3,
            Na=0,  # mM
            K=50,  # mM
            Tris=25,  # mM
            Mg=1.5,
            dnac1=500,  # nM
            dnac2=0,  # nM
            dNTPs=0.8,  # mM
            saltcorr=1,)    
    nebtm = int(nebtm)
    if nebtm != round(bptm):
        if nebtm > round(bptm):
            d = nebtm - 0.5 - bptm
        else:
            d = nebtm + 0.5 - bptm
        t.add_row([name, primer, nebtm, round(bptm, 4), round((abs(d)/bptm), 4)])
print(t)

+------+--------------------+-----+---------+--------+
| Name |      Sequence      | neb |    bp   | delta  |
+------+--------------------+-----+---------+--------+
| p142 | AATACAGTCTGCTACTTA |  48 | 48.5204 | 0.0004 |
| p154 | TTTATGTTACAACGAGAC |  49 | 48.4702 | 0.0006 |
|  p1  | CCCTCTCTTTTGGATGTA |  52 | 52.5062 | 0.0001 |
| p120 | ACTAGGGGTATTTCCAAC |  52 | 52.5352 | 0.0007 |
| p136 | TGTACGCACGTATAGTAC |  53 | 52.4891 | 0.0002 |
| p54  | TCAACTCTTGACAGCGAT |  55 | 55.5408 | 0.0007 |
| p57  | TAAGGGGCCCGTCTTAAA |  57 | 57.5968 | 0.0017 |
|  p7  | CTCACCTGGAGGTCCGGG |  63 | 63.5432 | 0.0007 |
+------+--------------------+-----+---------+--------+
